In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


In [2]:
data = pd.read_csv('train.csv',encoding = 'ISO-8859-1').set_index('ItemID')
test = pd.read_csv('test.csv',encoding = 'ISO-8859-1').set_index('ItemID')

# data = pd.read_csv('//Users//sharadjoshi//Desktop//Programming:Data Science//Kaggle data science//UMICH SI650 - Sentiment Classification//training.txt',encoding = 'ISO-8859-1')

In [3]:
# lowercase all data
data['SentimentText']=data['SentimentText'].str.lower()
test['SentimentText']=test['SentimentText'].str.lower()

In [4]:
# remove punctuation
table = str.maketrans('','',string.punctuation)
data['SentimentText'] = data['SentimentText'].apply(lambda x: x.translate(table))
test['SentimentText'] = test['SentimentText'].apply(lambda x: x.translate(table))

In [5]:
# tokenize words
data['SentimentText'] = data['SentimentText'].apply(nltk.word_tokenize)
test['SentimentText'] = test['SentimentText'].apply(nltk.word_tokenize)

In [6]:
# remove stopwords
stopword = stopwords.words('english')
for word in ['ain','aren','couldn','didn','doesn','hadn','hasn','haven','isn','mightn','mustn',
 'needn','shan','shouldn','wasn','weren','wouldn','don','no','nor','not','won']:
    stopword.remove(word)
data['SentimentText']=data['SentimentText'].apply(lambda x: [word for word in x if word not in stopword])
test['SentimentText']=test['SentimentText'].apply(lambda x: [word for word in x if word not in stopword])

In [7]:
# stemming
ps=PorterStemmer()
data['SentimentText']=data['SentimentText'].apply(lambda x: [ps.stem(word) for word in x])
test['SentimentText']=test['SentimentText'].apply(lambda x: [ps.stem(word) for word in x])

In [8]:
all_words=[]
data['SentimentText'].apply(lambda x: all_words.extend(x))

ItemID
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
30        None
          ... 
99971     None
99972     None
99973     None
99974     None
99975     None
99976     None
99977     None
99978     None
99979     None
99980     None
99981     None
99982     None
99983     None
99984     None
99985     None
99986     None
99987     None
99988     None
99989     None
99990     None
99991     None
99992     None
99993     None
99994     None
99995     None
99996     None
99997     None
99998     None
99999     None
100000    None
Name: SentimentText, Length: 99989, dtype: object

In [9]:
test.head()

,SentimentText
ItemID,
1,"[sad, apl, friend]"
2,"[miss, new, moon, trailer]"
3,"[omg, alreadi, 730]"
4,"[omgaga, im, sooo, im, gunna, cri, ive, dentis..."
5,"[think, mi, bf, cheat, tt]"


In [10]:
data['tweets'] = list(zip(data['SentimentText'],data['Sentiment']))
# test = data.iloc[1:30000]
# data = data.iloc[30000:]


In [11]:
word_features = nltk.FreqDist(all_words)
word_features_small = sorted(word_features,key=word_features.get,reverse=True)[:1000]

In [12]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features_small:
        features['contains(%s)' % word] = (word in document_words)
    return features


In [13]:
training_Set = nltk.classify.apply_features(extract_features, list(data['tweets']))

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_Set)

In [15]:
# classifier.show_most_informative_features(32)

In [ ]:
test['Sentiment'] = test['SentimentText'].apply(lambda x: classifier.classify(extract_features(x)))

In [ ]:
test[['ItemID','Sentiment']].to_csv('solution.csv')

In [18]:
# cm = test['Sentiment']-y_pred
# cm[cm==0].shape[0]/(cm[cm==0].shape[0]+cm[cm!=0].shape[0])